In [24]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
from skorch.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
from skorch import NeuralNetRegressor
import pickle

def identity_tokenizer(text):
    return text
def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

In [25]:
merged = pd.read_csv('mergedfullwithmoredates.csv')
from sklearn.model_selection import train_test_split  
mergedbig, mergedsmall = train_test_split(merged, test_size=0.15, random_state=0)
# mergedsmall.to_csv("mergedsmall.csv", index = False)
# mergedsmall = pd.read_csv('mergedsmall.csv')
merged = mergedsmall
merged.dropna()
# Obtaining the tweet contents into a list
all_tweets = merged["text"]
all_tweets = all_tweets.to_list()
import nltk
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
def lemming(tweet):
    a = word_tokenize(tweet)
    answer = list(map(lambda x: lemmatizer.lemmatize(x), a))
    return answer
from nltk.tokenize.treebank import TreebankWordDetokenizer
merged

,text,referenced_tweets,id,lang,Date,time,usedate,usedate1,usedate2,usedate3,...,Volume,Open,High,Low,Change,Change1,Change2,Change3,Change4,Change5
1147845,2020 is almost ended\n\nBRAND NEW SEALED FACTO...,1.317031e+18,1317047077411737601,en,2020-10-16,10:17:47,2020-10-16,2020-10-19,2020-10-20,2020-10-21,...,115393800.0,121.2800,121.5480,118.810,-2.260,-3.980,1.31,0.20,-1.7000,-1.3500
2318305,If you feel like Apple has no love for you by ...,NaN,1361013406829060106,en,2021-02-14,18:04:17,2021-02-16,2021-02-17,2021-02-18,2021-02-19,...,80576320.0,135.4900,136.0100,132.790,-2.300,-0.410,0.51,-0.37,-2.0100,2.1000
2772897,"📢NEW RIP IS LIVE📢\n\n2W1B 051 - iPhone, Capita...",NaN,1318150624626790402,en,2020-10-19,11:22:53,2020-10-19,2020-10-20,2020-10-21,2020-10-22,...,120639300.0,119.9600,120.4190,115.660,-3.980,1.310,0.20,-1.70,-1.3500,1.0400
1952614,#Apple has introduced a new #macOS version of ...,1.272588e+18,1272595399673630722,en,2020-06-15,18:22:41,2020-06-15,2020-06-16,2020-06-17,2020-06-18,...,138808920.0,83.3125,86.4200,83.145,2.435,0.155,-0.89,0.08,-1.2287,1.8825
33341,Good Deal😍 #MacBookPro #AppleEvent #Apple\n\nN...,1.389114e+18,1389114470304305154,en,2021-05-03,07:07:52,2021-05-03,2021-05-04,2021-05-05,2021-05-06,...,75135100.0,132.0400,134.0700,131.830,0.500,-3.340,-1.10,1.85,-0.6400,-2.5600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169957,Foreign Exchange History: How it All Started?\...,1.316240e+18,1316312507431051264,en,2020-10-14,09:38:52,2020-10-14,2020-10-15,2020-10-16,2020-10-19,...,151062300.0,121.0000,123.0300,119.620,0.190,1.990,-2.26,-3.98,1.3100,0.2000
2531839,"Tropicana Apple Juice, 10 oz., 24 Count\n\nSta...",NaN,1339593079318200320,en,2020-12-17,15:27:33,2020-12-17,2020-12-18,2020-12-21,2020-12-22,...,94359810.0,128.9000,129.5800,128.045,-0.200,-2.305,3.21,0.27,-1.2000,0.6500
1305376,"""emotional conclusion to a dazzling series!""\n...",1.303296e+18,1307074421098766336,en,2020-09-18,21:50:00,2020-09-18,2020-09-21,2020-09-22,2020-09-23,...,287104900.0,110.4000,110.8800,106.090,-3.560,5.540,-0.87,-4.50,3.0500,3.8500
1266310,3％超上昇した銘柄\n4434 サーバーワークス\n4625 アトミクス\n6777 ...,1.311853e+18,1311853293589725184,ja,2020-10-02,02:19:32,2020-10-02,2020-10-05,2020-10-06,2020-10-07,...,144712000.0,112.8900,115.3700,112.220,0.130,2.590,-2.54,0.46,-1.2800,1.6900


In [26]:
stop_words = set(stopwords.words('english'))
processed_tweets = []
X = all_tweets
for tweet in range(0, len(X)):  
    

    processed_tweet = re.sub(r"http\S+", "", str(X[tweet]))

    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', processed_tweet)
    
#     processed_tweet = re.sub(r'http\S+', '', processed_tweet)
    
    
#     processed_tweet = re.sub(r'co\S+', '', processed_tweet) 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
#     processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
    

    
    processed_tweet = re.sub(r'of|to|has|keep|ll', '', processed_tweet)
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
    
    processed_tweet = lemming(processed_tweet)
    
    processed_tweet = [word for word in processed_tweet if word not in stop_words]
    
    processed_tweet = TreebankWordDetokenizer().detokenize(processed_tweet)    
    
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
    processed_tweets.append(processed_tweet)

In [32]:
y = merged[['Change','Change1','Change2','Change3','Change4','Change5']]
# X = df3
# X = df2
# X = df2array
X = processed_tweets
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [33]:

lists = ['Change','Change1','Change2','Change3','Change4','Change5']
for pred in range(len(lists)):
    y_train2 = y_train.iloc[:,pred]
    y_test2 = y_test.iloc[:,pred]
    y_train2 = y_train2.values.reshape(-1,1)
    y_test2 = y_test2.values.reshape(-1,1)
    y_train2 = y_train2.astype(np.float32)
    y_test2 = y_test2.astype(np.float32)
    in_dimension = 5844
    hid_dimension = 10
    out_dimension = 1


    class PoleNN(nn.Module):
        def __init__(self):
            super(PoleNN, self).__init__()
            self.fc1 = nn.Linear(in_dimension,hid_dimension)
            self.fc2 = nn.Linear(hid_dimension,out_dimension)
            self.sigmoid = torch.nn.Sigmoid()

        def forward(self, X):
            hidden = self.fc1(X)
            hidden = self.sigmoid(hidden)
            output = self.fc2(hidden)
            return output

    from skorch import NeuralNetRegressor
    from sklearn.pipeline import Pipeline
    x_trainshape = 5844
    class RegressorModule(nn.Module):
        def __init__(
                self,
                num_units=10,
                nonlin=F.relu,
        ):
            super(RegressorModule, self).__init__()
            self.num_units = num_units
            self.nonlin = nonlin

            self.dense0 = nn.Linear(x_trainshape, num_units)
            self.nonlin = nonlin
            self.dense1 = nn.Linear(num_units, 10)
            self.output = nn.Linear(10, 1)

        def forward(self, X, **kwargs):
            X = self.nonlin(self.dense0(X))
            X = F.relu(self.dense1(X))
            X = self.output(X)
            return X

    pole_model = RegressorModule()
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(pole_model.parameters(), lr = 0.1)

    net = NeuralNetRegressor(module=pole_model, max_epochs=10, lr=0.1, callbacks =[('earlystopping',EarlyStopping())])
    def typechange(x):
        return x.astype(dtype = np.float32)
    typetransform = FunctionTransformer(typechange)
    def inputneuron(x):
        x_trainshape = x.shape[1]
    #     return x_trainshape
    inputneuronnumber = FunctionTransformer(inputneuron)
    # pipe = Pipeline([('transform', DataFrameTransformer()),('net', net)])
    # pipe = Pipeline([('net', net)])
    pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True,token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("net", net)])


    # net = skorch.NeuralNetClassifier(module=PoleNN, max_epochs=20, lr=0.1, criterion=torch.nn.NLLLoss)
    pipe.fit(X=x_train, y=y_train2)
    y_pred = pipe.predict(x_test)
    rmse = mean_squared_error(y_test2, y_pred, squared = False)
    print(lists[pred], "RMSE:", rmse)

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1        4.1885        4.1607  75.4138
      2        4.0550        4.1116  71.8156
      3        4.0135        4.0859  72.1005
      4        3.9949        4.0711  70.5459
      5        3.9805        4.0484  69.9495
      6        3.9659        4.0260  70.4210
      7        3.9482        3.9760  70.6778
      8        3.9278        3.9615  70.7877
      9        3.9142        3.9526  69.0504
     10        3.9045        3.9453  74.5129
Change RMSE: 1.9767237
  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1           nan           nan  69.1494
      2           nan           nan  69.5809
      3           nan           nan  69.0235
      4           nan           nan  69.0874
Stopping since valid_loss has not improved in the last 5 epochs.


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').